# All is Graph

Reference:
1. [图卷积网络到底怎么做，这是一份极简的Numpy实现](https://mp.weixin.qq.com/s/sg9O761F0KHAmCPOfMW_kQ)

## Examples

![Cited from [图卷积网络到底怎么做，这是一份极简的Numpy实现](https://mp.weixin.qq.com/s/sg9O761F0KHAmCPOfMW_kQ)](https://mmbiz.qpic.cn/mmbiz_png/KmXPKA19gW8VBwmtdqsNQQEicrmm06aF2OhU65hickwqmthgLOurE3rpdE2MhZhWo7PzfOK4nz3CicDIyfxv8Wl6Q/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

## GCN

In [1]:
import torch
import numpy as np

### 构建邻接矩阵

In [6]:
A = np.matrix([[0, 1, 0, 1],
               [1, 0, 1, 1],
               [0, 1, 0, 1],
               [1, 1, 1, 0]], dtype=float)

In [7]:
node_features = np.random.rand(4, 6)
node_features

array([[0.73166656, 0.97124384, 0.39010042, 0.35377135, 0.60371244,
        0.10047358],
       [0.19250475, 0.19464946, 0.74503961, 0.15040043, 0.15262087,
        0.89864691],
       [0.62822714, 0.90285514, 0.08829209, 0.65172925, 0.70519149,
        0.47095836],
       [0.29454457, 0.67208882, 0.57597568, 0.7861017 , 0.84657429,
        0.10542533]])

In [8]:
A * node_features

matrix([[0.48704932, 0.86673827, 1.32101529, 0.93650213, 0.99919516,
         1.00407224],
        [1.65443826, 2.54618779, 1.05436819, 1.7916023 , 2.15547822,
         0.67685727],
        [0.48704932, 0.86673827, 1.32101529, 0.93650213, 0.99919516,
         1.00407224],
        [1.55239845, 2.06874843, 1.22343212, 1.15590102, 1.4615248 ,
         1.47007885]])

### 由于原始邻接矩阵上的节点与节点特征矩阵相乘后对自己没有表征，因此加上一个单位矩阵使邻接矩阵对自己有自链接

In [9]:
I = np.eye(A.shape[0])
I

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [10]:
A_hat = A + I
A_hat * node_features

matrix([[1.21871588, 1.83798211, 1.71111571, 1.29027348, 1.6029076 ,
         1.10454582],
        [1.84694301, 2.74083725, 1.7994078 , 1.94200273, 2.30809909,
         1.57550418],
        [1.11527646, 1.76959341, 1.40930738, 1.58823138, 1.70438665,
         1.4750306 ],
        [1.84694301, 2.74083725, 1.7994078 , 1.94200273, 2.30809909,
         1.57550418]])

### 也可以考虑使用拉普拉斯矩阵，即$L=A-D$ 

In [19]:
A

matrix([[0., 1., 0., 1.],
        [1., 0., 1., 1.],
        [0., 1., 0., 1.],
        [1., 1., 1., 0.]])

In [20]:
D = np.array(np.sum(A, axis=0))[0]
D = np.matrix(np.diag(D))

In [22]:
D_hat = np.array(np.sum(A, axis=0))[0]
D_hat = np.matrix(np.diag(D))
L = A - D
L

matrix([[-2.,  1.,  0.,  1.],
        [ 1., -3.,  1.,  1.],
        [ 0.,  1., -2.,  1.],
        [ 1.,  1.,  1., -3.]])

In [24]:
A

matrix([[0., 1., 0., 1.],
        [1., 0., 1., 1.],
        [0., 1., 0., 1.],
        [1., 1., 1., 0.]])

In [26]:
np.linalg.pinv(A) * A

matrix([[ 5.00000000e-01,  5.55111512e-17,  5.00000000e-01,
          5.55111512e-17],
        [-5.55111512e-17,  1.00000000e+00, -5.55111512e-17,
          1.11022302e-16],
        [ 5.00000000e-01,  1.66533454e-16,  5.00000000e-01,
          1.11022302e-16],
        [ 0.00000000e+00, -3.33066907e-16,  0.00000000e+00,
          1.00000000e+00]])

### 归一化处理

In [11]:
# 求出度矩阵（已经算上自链接）
D_hat = np.array(np.sum(A+I, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))
D_hat

matrix([[2., 0., 0., 0.],
        [0., 3., 0., 0.],
        [0., 0., 2., 0.],
        [0., 0., 0., 3.]])

In [12]:
((D_hat**-1)*A_hat*node_features).shape

(4, 6)

In [14]:
input_size = node_features.shape[1]
output_size = 12
# 初始化权重矩阵
w = np.random.rand(input_size, output_size)

In [16]:
layer1 = D_hat**-1*A_hat*node_features * w

In [17]:
layer1.shape

(4, 12)

### PyTorch geometric
![](https://pytorch-geometric.readthedocs.io/en/latest/_images/graph.svg)
* [pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html)
* [图神经网络库PyTorch geometric（PYG）零基础上手教程](https://zhuanlan.zhihu.com/p/91229616)

In [1]:
import torch
from torch_geometric.data import Data

In [3]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index.t().contiguous())
data

Data(x=[3, 1], edge_index=[2, 4])

# HeteGraph

In [2]:
import pandas as pd

In [2]:
# u, i, r
u = [0, 1, 2, 3]
s = [4, 5, 6]

edges = [
    [0, 4, 2],
    [0, 5, 3],
    [1, 4, 1],
    [1, 6, 2],
    [2, 4, 1],
    [3, 5, 2],
    [3, 6, 2]
]

ux = np.matrix([[i, -i] for i in range(len(u))])
sx = np.matrix([[i, -i] for i in range(len(s))])

A = np.matrix(
    [
        [0, 0, 0, 0, 2, 3, 0],
        [0, 0, 0, 0, 1, 0, 2],
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 2, 2],
        [2, 1, 1, 0, 0, 0, 0],
        [3, 0, 0, 2, 0, 0, 0],
        [0, 2, 0, 2, 0, 0, 0]
    ]
)

In [3]:
I = np.eye(A.shape[0])
A_hat = A + I
A_hat

matrix([[1., 0., 0., 0., 2., 3., 0.],
        [0., 1., 0., 0., 1., 0., 2.],
        [0., 0., 1., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 2., 2.],
        [2., 1., 1., 0., 1., 0., 0.],
        [3., 0., 0., 2., 0., 1., 0.],
        [0., 2., 0., 2., 0., 0., 1.]])

In [4]:
D = np.array(np.sum(A_hat, axis=0))[0]
D = np.matrix(np.diag(D))
D

matrix([[6., 0., 0., 0., 0., 0., 0.],
        [0., 4., 0., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0., 0.],
        [0., 0., 0., 5., 0., 0., 0.],
        [0., 0., 0., 0., 5., 0., 0.],
        [0., 0., 0., 0., 0., 6., 0.],
        [0., 0., 0., 0., 0., 0., 5.]])

In [7]:
A_hat.shape

(7, 7)

In [6]:
(D**-1).shape

(7, 7)

In [8]:
(D**-1) * A_hat

: 

: 

## LightGCN

## UltraGCN